In [15]:
import pandas as pd
from sklearn.metrics import accuracy_score ,  roc_curve, auc , classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
import xgboost as xgb 
import statistics
from scipy.fftpack import fft, ifft

!pip install biopython
import Bio

from itertools import product

import collections

from imblearn.over_sampling import SMOTE

import operator

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Record Constants
INPUT_TRAIN_IN = "train_in_small.csv"
INPUT_TRAIN_OUT = "train_out_small.csv"  
INPUT_TEST_IN = "test_in.csv"
INPUT_TEST_OUT = "test_out.csv"

WINDOW_SIZE =  100 # Final RNA Sequence will be 201 Length 
MAX_LENGTH = (WINDOW_SIZE * 2)+1

In [3]:
#Read X Varaibles and Y Varaibles

x_train_raw =  pd.read_csv( INPUT_TRAIN_IN, header=None , skiprows=1 )
y_train_raw =  pd.read_csv(  INPUT_TRAIN_OUT, header=None , skiprows=1 )

x_test_raw =  pd.read_csv( INPUT_TEST_IN, header=None , skiprows=1)
y_test_raw =  pd.read_csv( INPUT_TEST_OUT, header=None , skiprows=1)

In [4]:
def des_start_code(codes):
	if codes in ('ATG', 'AUG'):
		return True
	return False

In [5]:
def des_end_code(codes):
	if codes in ('TAA', 'UAA', 'TAG', 'UAG', 'TGA', 'UGA'):
		return True
	return False

In [6]:
def read_by_three(string, offset):
	flag = True
	length = len(string)
	start = end = -1
	i = 0
	result = set()
	while i < length-2:
		codes = string[i:i+3]
		if des_start_code(codes) and flag:
			start = i
			flag = False
		if des_end_code(codes) and not flag:
			end = i + 2
			flag = True
		if (end > start) and (start != -1):
			result.add((start + offset, end + offset))
		i = i + 3
	return result

In [7]:
def get_gc(string):
	gc = ((string.count('G') + string.count('C')) / len(string)) * 100
	return gc

In [8]:
def get_info(string, pos):
	length = pos[1] - pos[0] + 1
	gc = get_gc(string[pos[0]:pos[1]+1])
	return str(pos[0]), str(pos[1]), str(length), str(gc)

In [9]:
def orf(seq):
	result_info = []
	strings = [seq, seq[1:], seq[2:]]
	for index, string in enumerate(strings):
		# print(index)
		# print(string)
		positions = read_by_three(string, index)
		positions = sorted(positions, key=operator.itemgetter(0))
		# print(positions)
		for pos in positions:
			result_info.append(get_info(seq, pos))
	# print(result_info)
	# print(len(result_info))
	return result_info

In [10]:
def getExactPos(x_train_raw):
    middle_index = (x_train_raw.shape[1] // 2) + 1
    truncated_df = x_train_raw.iloc[:, middle_index - WINDOW_SIZE - 1: middle_index + WINDOW_SIZE]
    return truncated_df 

x_train_split = getExactPos(x_train_raw)
x_train_split

,400,401,402,403,404,405,406,407,408,409,...,591,592,593,594,595,596,597,598,599,600
0,G,G,G,C,G,C,C,T,C,G,...,C,C,T,C,T,C,C,T,A,G
1,A,A,C,A,G,T,A,A,A,C,...,T,T,T,T,T,T,T,T,T,C
2,A,A,A,G,A,A,C,A,T,G,...,C,A,G,A,A,T,T,A,G,G
3,G,G,G,T,G,G,T,G,C,C,...,G,T,C,T,C,T,A,A,G,A
4,T,C,A,T,T,G,A,A,G,T,...,A,A,A,T,A,T,G,C,T,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1408,A,T,T,T,G,A,T,G,T,T,...,T,G,T,A,T,A,A,T,A,N
1409,G,A,G,T,T,G,A,G,T,G,...,G,T,T,A,T,G,A,G,A,N
1410,C,T,C,A,C,A,G,G,A,A,...,T,C,T,G,T,G,C,T,A,N
1411,G,T,T,T,T,T,A,A,A,C,...,T,C,G,G,C,T,T,A,N,N


In [13]:
# x_train_split.to_csv("out.csv")

x_train_combined =  pd.read_csv( "out.csv", header=None , skiprows=1 )

x_train_combined

,0
0,GGGCGCCTCGGCTCTTGTCCACGCCCTGGACCGCGGGGACCGCGAG...
1,AACAGTAAACCAAATGCATTGTAACATCTTATTTAATTCTAACATG...
2,AAAGAACATGCTTCTTCCTTCTTAGTATATAGATCAGGATCCACTT...
3,GGGTGGTGCCTGGACAGAGTGAGTGTCTAATGATTGCTAAAGTAAT...
4,TCATTGAAGTTTTTTTGATCACAGGAATAACATGTTTAGATAGTAA...
...,...
1408,ATTTGATGTTCACCTTTTATACCATAATCCTGTTCATTGAAAAACA...
1409,GAGTTGAGTGAATCCAGGTGAGTACGGATGTTGCTCCTGCCCCTTC...
1410,CTCACAGGAAATGGTTCCCAAGGCTATGCCTGAGATAAGGCTGGCT...
1411,GTTTTTAAACTTTTCTAAAATATTACTTACCAAAATATTTTCCTCT...


In [18]:
result = x_train_combined.apply(orf, axis=1)
result

0       []
1       []
2       []
3       []
4       []
        ..
1408    []
1409    []
1410    []
1411    []
1412    []
Length: 1413, dtype: object

In [17]:
# result.to_csv("output.csv")